In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "4"
os.environ["MKL_NUM_THREADS"] = "4"
os.environ["MKL_DYNAMIC"] = "FALSE"
from threadpoolctl import threadpool_limits, threadpool_info

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import datacompy
import os, sys
import numpy as np
import re
import ast
from pathlib import Path
import openpyxl
import itertools
import torch

# narzedzia
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    cross_val_predict,
    learning_curve,
    RepeatedStratifiedKFold,
    GridSearchCV,
    RandomizedSearchCV
)
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix, silhouette_score, accuracy_score, pairwise_distances, make_scorer, precision_score, f1_score, roc_auc_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import tree
from sklearn.tree import plot_tree
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from scipy import stats
from scipy.stats import chi2_contingency, f_oneway, friedmanchisquare, wilcoxon
from scipy.spatial import distance
from joblib import dump, load

# modele
from xgboost import XGBClassifier, XGBRFClassifier
#from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# methods
from imblearn.under_sampling import ClusterCentroids, NearMiss
from scipy.optimize import differential_evolution
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, HDBSCAN
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor, KNeighborsClassifier, NearestNeighbors
from sklearn.inspection import permutation_importance
from scipy.spatial.distance import euclidean
from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from ctgan import CTGAN

from threadpoolctl import threadpool_limits, threadpool_info


Please note that you are missing the optional dependency: fugue. If you need to use this functionality it must be installed.
Please note that you are missing the optional dependency: snowflake. If you need to use this functionality it must be installed.
Please note that you are missing the optional dependency: spark. If you need to use this functionality it must be installed.
Python 3.12 and above currently is not supported by Spark and Ray. Please note that some functionality will not work and currently is not supported.


### Read Datasets

In [2]:
borderline_df = pd.read_parquet("D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\borderline_data.parquet") 
borderline2_df = pd.read_parquet("D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\borderline3_data.parquet")
smote_df = pd.read_parquet("D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\smote_data.parquet")
smote2_df = pd.read_parquet("D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\smote3_data.parquet")
gan_df = pd.read_parquet("D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\GAN_data.parquet")
gan2_df = pd.read_parquet("D:\\ml\\undersampling_data\\data\\cicunsw\\oversampling\\GAN3_data.parquet")

original_df = pd.read_parquet("D:\\ml\\undersampling_data\\data\\cicunsw\\CICUNSW_train.parquet")

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\ml\\undersampling_data\\data\\cicunsw\\CICUNSW_train.parquet'

In [ ]:
print(datacompy.Compare(original_df, borderline_df, join_columns='Label').report())

In [ ]:
borderline_df.drop_duplicates(inplace=True)

In [ ]:
original_df["Label"].value_counts().sum()

In [ ]:
print(smote_df["Label"].value_counts())

In [ ]:
print(smote_df["Label"].value_counts())
print(borderline_df["Label"].value_counts())

In [ ]:
mix_df = pd.concat([borderline_df, borderline2_df, smote_df, smote2_df, gan_df, gan2_df], ignore_index=True)
smote_df = pd.concat([smote_df, smote2_df], ignore_index=True)
gan_df = pd.concat([gan_df, gan2_df], ignore_index=True)
borderline_df = pd.concat([borderline_df, borderline2_df], ignore_index=True)

In [ ]:
print(original_df["Label"].value_counts())
count1=original_df["Label"].value_counts().sum()
count2=abs((original_df['Label']==0).sum() - (original_df['Label']==1).sum())
print(f"Before undersampling: {count2}")
print(f"After number of samples: {count1+count2}")

Label
0    2415460
1      62708
Name: count, dtype: int64
Before undersampling: 2352752
After number of samples: 4830920


In [ ]:
#Split data
X_mix, y_mix = mix_df.drop(columns=["Label", "source"]), mix_df["Label"]
X_smote, y_smote = smote_df.drop(columns=["Label", "source"]), smote_df["Label"]
X_GAN, y_GAN = gan_df.drop(columns=["Label", "source"]), gan_df["Label"]
X_borderline, y_borderline = borderline_df.drop(columns=["Label", "source"]), borderline_df["Label"]

#Dictionary
data = {}
data["mix"] = (X_mix, y_mix)
data["smote"] = (X_smote, y_smote)
data["GAN"] = (X_GAN, y_GAN)
data["borderline"] = (X_borderline, y_borderline)

compare = {}
compare["mix"] = mix_df
compare["smote"] = smote_df
compare["GAN"] = gan_df
compare["borderline"] = borderline_df

### Undersampling

In [ ]:
base = Path("D:/ml/undersampling_data/data/cicunsw/reduced")
#base2 = Path("D:/ml/undersampling_data/models/kmeans&centroids2")

df_ = {}

for name in data.keys() & compare.keys():
    # Read data from dictionary
    X_train, y_train = data[name]
    compare_df = compare[name]
    
    # Select majority class
    X_majority = X_train[y_train == 1]
    
    KM = KMeans(n_clusters=(int)(count2))
    with threadpool_limits(limits=16):
    # Apply KMeans clustering
        kmeans = KM.fit(X_majority)
    
    # Create a DataFrame for centroids
    X_majority_reduced = pd.DataFrame(kmeans.cluster_centers_, columns=X_train.columns)
    y_majority_reduced = pd.Series([1] * (int)(count2), name="Label") 
    
    # Combine reduced majority class with original minority class
    df_majority = pd.concat([X_majority_reduced, y_majority_reduced], axis=1).reset_index(drop=True)
    
    # Add source column if not present
    df_majority["source"] = None
    missing_source = df_majority[df_majority["source"].isna()]
    if not missing_source.empty:
        df_majority.loc[df_majority["source"].isna(), "source"] = "centroid" 
    
    print(df_majority)
    df_majority = df_majority.reindex(columns=original_df.columns, fill_value=0.0)
    
    # Combine with original minority class
    df_[name] = pd.concat([df_majority, original_df], axis=0).reset_index(drop=True)  
    
    print(df_[name].info())
    
    # Save to CSV if file does not exist
    file_path = base / f"{name}_KM_centroids.csv"

    if os.path.exists(file_path):
        print("File exists.")
    else:
        df_[name].to_csv(file_path, index=False)
        print("File saved.")

In [ ]:
try:
    import cudf
    import cupy as cp
    from cuml.cluster import KMeans as cuKMeans
    GPU_AVAILABLE = True
except Exception as e:
    GPU_AVAILABLE = False
    GPU_IMPORT_ERROR = e

# --- CPU fallback (jeśli chcesz, by kod działał także bez GPU) ---
if not GPU_AVAILABLE:
    import pandas as pd
    from sklearn.cluster import KMeans as skKMeans
else:
    import pandas as pd  # użyjemy do I/O i finalnego reindexu

for name in data.keys() & compare.keys():
    # --- Wejście z dictów ---
    X_train, y_train = data[name]      # pandas DataFrame/Series
    compare_df = compare[name]         # nieużywane dalej

    # --- Selekcja klasy większości: Label == 1 ---
    #   Zakładam: y_train == 1 oznacza większość (jak w Twoim kodzie)
    #   Jeśli odwrotnie, podmień warunek.
    X_majority_pd = X_train[y_train == 1]

    if GPU_AVAILABLE:
        # ==== GPU ŚCIEŻKA ====
        # Przenieś dane na GPU:
        X_majority = cudf.from_pandas(X_majority_pd)

        # KMeans na GPU:
        KM = cuKMeans(
            n_clusters=int(count2),
            init="k-means++",
            max_iter=300,
            random_state=42,
            verbose=0,
        )
        kmeans = KM.fit(X_majority)

        # Centroidy: cupy ndarray -> cuDF -> pandas
        centers_cu = kmeans.cluster_centers_           # cupy.ndarray
        X_majority_reduced_gdf = cudf.DataFrame(centers_cu, columns=X_train.columns)
        X_majority_reduced = X_majority_reduced_gdf.to_pandas()

    else:
        # ==== CPU FALLBACK ====
        KM = skKMeans(
            n_clusters=int(count2),
            init="k-means++",
            max_iter=300,
            n_init="auto",
            random_state=42,
            verbose=0,
        )
        kmeans = KM.fit(X_majority_pd)
        X_majority_reduced = pd.DataFrame(
            kmeans.cluster_centers_,
            columns=X_train.columns
        )

    # Etykiety centroidów (większość)
    y_majority_reduced = pd.Series([1] * int(count2), name="Label")

    # Zbuduj df dla większości (centroidy + Label)
    df_majority = pd.concat([X_majority_reduced, y_majority_reduced], axis=1).reset_index(drop=True)

    # Kolumna 'source'
    if "source" not in df_majority.columns:
        df_majority["source"] = None
    df_majority.loc[df_majority["source"].isna(), "source"] = "centroid"

    # Dopasuj kolumny do original_df
    df_majority = df_majority.reindex(columns=original_df.columns, fill_value=0.0)

    # Połącz z mniejszością (original_df)
    df_[name] = pd.concat([df_majority, original_df], axis=0).reset_index(drop=True)

    # Info i zapis
    print(df_[name].info())

    file_path = base / f"{name}_KM_centroids.csv"
    if os.path.exists(file_path):
        print(f"File exists: {file_path}")
    else:
        df_[name].to_csv(file_path, index=False)
        print(f"File saved: {file_path}")

# (opcjonalnie) komunikat jeśli szedłeś fallbackiem:
if not GPU_AVAILABLE:
    print(
        "\n[WARN] Uruchomiono CPU fallback (cuDF/cuML niedostępne). "
        f"Import error: {GPU_IMPORT_ERROR}"
    )